In [1]:
# Dependencies
import pymongo
import datetime
import pandas as pd
import re

In [2]:
# Connect to database
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['carsguide_used_cars_australia']
collection = db.scraped_cars

# Create df
df = pd.DataFrame(list(collection.find()))
df.head()

,_id,Dealer Name,Year-make-model,KMs,Tansmission,Price,Comments,Source URL,Year,Make,Model,State Abrev,Suburb,Listing Type,Date scraped,Page Number
0,6163e5a9b8a22997ab97c3d4,Carvana,2015 Porsche Macan S Diesel,85379,Automatic,68988,Sports automatic 3.0ltr turbo diesel Macan wag...,https://www.carsguide.com.au/buy-a-car/all-new...,2015,Porsche,"[Macan, S, Diesel]",WA,KINGSLEY,USED Dealer ad,2021/10/11,0
1,6163e5a9b8a22997ab97c3d5,Aberdeen Auto Auctions,2012 Hyundai Elantra Active,201522,Automatic,7990,Be quick for this one owner vehicle complete w...,https://www.carsguide.com.au/buy-a-car/all-new...,2012,Hyundai,"[Elantra, Active]",WA,West Perth,USED Dealer ad,2021/10/11,0
2,6163e5a9b8a22997ab97c3d6,Prindiville Car Sales,2011 Hyundai I20 Active,109575,Automatic,10990,Comprehensive warranty options and competitive...,https://www.carsguide.com.au/buy-a-car/all-new...,2011,Hyundai,"[I20, Active]",WA,Wangara,USED Dealer ad,2021/10/11,0
3,6163e5a9b8a22997ab97c3d7,Canning Vale Toyota Pre-Owned,2019 Toyota Granvia,260,NULL,55888,Toyota Granvia has just arrived with Full-Serv...,https://www.carsguide.com.au/buy-a-car/all-new...,2019,Toyota,"[Granvia, ]",WA,Canning Vale,USED Dealer ad,2021/10/11,0
4,6163e5a9b8a22997ab97c3d8,Affordable Cars Wangara,2013 Holden Captiva 7 SX (fwd),219918,Automatic,7888,Call Lee 0422042409,https://www.carsguide.com.au/buy-a-car/all-new...,2013,Holden,"[Captiva, 7, SX, (fwd)]",WA,Wangara,USED Dealer ad,2021/10/11,0


In [3]:
# Clean Data
df = df[df.Price != 'POA']
df['Price'] = df.loc[:,"Price"].astype(float)
df = df[df.Price > 500]
df = df[df.Price < 900000]
df = df[df.Price != 1234]
df = df[df['Dealer Name'] != 'POA']
df = df[df['State Abrev'] != 'Hyundai']
df.head()

,_id,Dealer Name,Year-make-model,KMs,Tansmission,Price,Comments,Source URL,Year,Make,Model,State Abrev,Suburb,Listing Type,Date scraped,Page Number
0,6163e5a9b8a22997ab97c3d4,Carvana,2015 Porsche Macan S Diesel,85379,Automatic,68988.0,Sports automatic 3.0ltr turbo diesel Macan wag...,https://www.carsguide.com.au/buy-a-car/all-new...,2015,Porsche,"[Macan, S, Diesel]",WA,KINGSLEY,USED Dealer ad,2021/10/11,0
1,6163e5a9b8a22997ab97c3d5,Aberdeen Auto Auctions,2012 Hyundai Elantra Active,201522,Automatic,7990.0,Be quick for this one owner vehicle complete w...,https://www.carsguide.com.au/buy-a-car/all-new...,2012,Hyundai,"[Elantra, Active]",WA,West Perth,USED Dealer ad,2021/10/11,0
2,6163e5a9b8a22997ab97c3d6,Prindiville Car Sales,2011 Hyundai I20 Active,109575,Automatic,10990.0,Comprehensive warranty options and competitive...,https://www.carsguide.com.au/buy-a-car/all-new...,2011,Hyundai,"[I20, Active]",WA,Wangara,USED Dealer ad,2021/10/11,0
3,6163e5a9b8a22997ab97c3d7,Canning Vale Toyota Pre-Owned,2019 Toyota Granvia,260,NULL,55888.0,Toyota Granvia has just arrived with Full-Serv...,https://www.carsguide.com.au/buy-a-car/all-new...,2019,Toyota,"[Granvia, ]",WA,Canning Vale,USED Dealer ad,2021/10/11,0
4,6163e5a9b8a22997ab97c3d8,Affordable Cars Wangara,2013 Holden Captiva 7 SX (fwd),219918,Automatic,7888.0,Call Lee 0422042409,https://www.carsguide.com.au/buy-a-car/all-new...,2013,Holden,"[Captiva, 7, SX, (fwd)]",WA,Wangara,USED Dealer ad,2021/10/11,0


In [4]:
# Checking for anomolies and outliers
df.sort_values(by=['Price'])

,_id,Dealer Name,Year-make-model,KMs,Tansmission,Price,Comments,Source URL,Year,Make,Model,State Abrev,Suburb,Listing Type,Date scraped,Page Number
319,6163e642b8a22997ab97c513,TM Wholesale,2005 Holden Astra Classic,42597,Automatic,6900.0,"Low Kms Original 42,857 Kms auto transmission ...",https://www.carsguide.com.au/buy-a-car/all-new...,2005,Holden,"[Astra, Classic]",WA,Mount Lawley,USED Dealer ad,2021/10/11,34
386,6163e667b8a22997ab97c556,DJ Autos pty ltd,2010 Skoda Octavia 1.8 TSI,128272,Automatic,7888.0,*** Stunning Fastback Skoda Fulk log book serv...,https://www.carsguide.com.au/buy-a-car/all-new...,2010,Skoda,"[Octavia, 1.8, TSI]",WA,Bellevue,USED Dealer ad,2021/10/11,43
4,6163e5a9b8a22997ab97c3d8,Affordable Cars Wangara,2013 Holden Captiva 7 SX (fwd),219918,Automatic,7888.0,Call Lee 0422042409,https://www.carsguide.com.au/buy-a-car/all-new...,2013,Holden,"[Captiva, 7, SX, (fwd)]",WA,Wangara,USED Dealer ad,2021/10/11,0
1,6163e5a9b8a22997ab97c3d5,Aberdeen Auto Auctions,2012 Hyundai Elantra Active,201522,Automatic,7990.0,Be quick for this one owner vehicle complete w...,https://www.carsguide.com.au/buy-a-car/all-new...,2012,Hyundai,"[Elantra, Active]",WA,West Perth,USED Dealer ad,2021/10/11,0
131,6163e5e1b8a22997ab97c457,Car Hub WA,2013 Ford Fiesta,139032,Manual,7990.0,2013 Ford Fiesta White 5 Speed Manual Hatchback,https://www.carsguide.com.au/buy-a-car/all-new...,2013,Ford,"[Fiesta, ]",WA,Yangebup,USED Dealer ad,2021/10/11,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,6163e5a9b8a22997ab97c3dc,Barbagallo Prestige Used,2019 BMW Z4 M40i Steptronic,17128,NULL,114990.0,2019 BMW Z4 M40IFinished in San Francisco red ...,https://www.carsguide.com.au/buy-a-car/all-new...,2019,BMW,"[Z4, M40i, Steptronic]",WA,Osborne Park,USED Dealer ad,2021/10/11,0
14,6163e5b1b8a22997ab97c3e2,Midland Mitsubishi Used Cars,2021 Porsche Macan S,16873,Automatic,117990.0,PERFECT EXAMPLE OF LUXURY AND CLASS!!!! NEAR N...,https://www.carsguide.com.au/buy-a-car/all-new...,2021,Porsche,"[Macan, S]",WA,Midland,USED Dealer ad,2021/10/11,1
7,6163e5a9b8a22997ab97c3db,Osborne Park Mazda used,2017 Mercedes-Benz GLE63 S 4Matic,69606,Automatic,124888.0,**IF YOU CAN SEE THIS AD IM AVAILABLE!!! **LOC...,https://www.carsguide.com.au/buy-a-car/all-new...,2017,Mercedes-Benz,"[GLE63, S, 4Matic]",WA,Osborne Park,USED Dealer ad,2021/10/11,0
250,6163e623b8a22997ab97c4ce,Canning Vale Toyota Pre-Owned,2018 HSV Colorado Sportscat Plus (4X4),24821,Automatic,125000.0,HSV COLORADO SPORTSCAT+ LSA CONVERSION & TRANS...,https://www.carsguide.com.au/buy-a-car/all-new...,2018,HSV,"[Colorado, Sportscat, Plus, (4X4)]",WA,Canning Vale,USED Dealer ad,2021/10/11,26


In [7]:
# Number of listings per Suburb
no_per_suburb = df['Suburb'].value_counts()

In [ ]:
# Work out average Price by Suburb
# pd.options.display.float_format = '${:,.2f}'.format
df_state_grouped = df.loc[:,['State Abrev', 'Suburb', 'Price','KMs', 'Comments']]
df_state_grouped = df_state_grouped.groupby(['State Abrev','Suburb'])
df_state_grouped['Price'].mean().round()